# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Backfill Features to the Feature Store</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/{project_name}/{notebook_name}.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Fetch historical data
2. Connect to the Hopsworks feature store
3. Create feature groups and insert them to the feature store

![tutorial-flow](../../images/01_featuregroups.png)

## <span style='color:#ff5f27'> 📝 Imports

In [1]:
import pandas as pd

from functions import *

---

## <span style='color:#ff5f27'> 💽 Loading Historical Data</span>


#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [3]:
df_air_quality = pd.read_csv('https://repo.hops.works/dev/davit/air_quality/air_quality.csv')
df_air_quality.head()

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,pm10_avg,pm10_max,pm10_min,pm25_avg,pm25_max,pm25_min,uvi_avg,uvi_max,uvi_min
0,Sundsvall,11,2022-09-06,34.30,1020.5,10,15.80,19,25,13,3,4,2,7,8,6,0,0,0
1,Kyiv,6,2022-09-06,98.78,1022.7,2,13.91,19,27,11,7,11,4,23,39,14,0,0,0
2,Stockholm,13,2022-09-06,59.00,1021.0,13,17.00,17,25,6,6,12,3,14,30,9,0,0,0
3,Malmo,23,2022-09-06,54.50,1019.0,4,17.30,27,32,24,5,5,3,10,11,8,1,1,1
4,Stockholm,1,2022-09-07,46.00,1022.0,1,16.00,18,26,8,7,10,4,16,21,8,1,1,1


In [9]:
df_air_quality.city.unique()

array(['Kyiv', 'Malmo', 'Stockholm', 'Sundsvall'], dtype=object)

In [4]:
df_air_quality.date = df_air_quality.date.apply(timestamp_2_time)
df_air_quality.sort_values(by = ['city','date'],inplace = True,ignore_index = True)

df_air_quality.head()

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,pm10_avg,pm10_max,pm10_min,pm25_avg,pm25_max,pm25_min,uvi_avg,uvi_max,uvi_min
0,Kyiv,6,1662415200000,98.78,1022.7,2,13.91,19,27,11,7,11,4,23,39,14,0,0,0
1,Kyiv,2,1662501600000,65.58,1020.0,1,21.02,22,32,10,9,12,5,29,43,15,0,0,0
2,Kyiv,4,1662588000000,99.90,1017.6,2,12.45,22,32,10,9,12,5,29,43,15,0,0,0
3,Kyiv,2,1662674400000,99.90,1021.3,1,10.00,20,33,6,10,17,5,33,56,16,0,0,0
4,Malmo,23,1662415200000,54.50,1019.0,4,17.30,27,32,24,5,5,3,10,11,8,1,1,1


#### <span style='color:#ff5f27'> 🌦 Weather Data

In [5]:
df_weather = pd.read_csv('https://repo.hops.works/dev/davit/air_quality/weather.csv')

df_weather.head(3)

,city,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,Kyiv,2022-09-06,17.7,4.6,11.5,17.7,4.6,11.5,1.8,55.3,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
1,Sundsvall,2022-09-06,13.0,3.0,8.6,13.0,0.1,7.4,5.5,81.8,...,31.0,14.3,192.9,1024.1,90.8,15.3,116.1,10.1,5.0,Overcast
2,Stockholm,2022-09-06,15.9,7.8,12.0,15.9,7.1,11.8,7.0,73.6,...,25.2,13.0,70.7,1022.0,59.5,15.3,132.5,11.6,5.0,Partially cloudy


In [6]:
df_weather.date = df_weather.date.apply(timestamp_2_time)
df_weather.sort_values(by=['city', 'date'],inplace=True, ignore_index=True)

df_weather.head(3)

,city,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,Kyiv,1662415200000,17.7,4.6,11.5,17.7,4.6,11.5,1.8,55.3,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
1,Kyiv,1662501600000,17.7,4.6,11.5,17.7,4.6,11.5,1.8,55.3,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
2,Kyiv,1662588000000,21.1,7.9,14.3,21.1,7.9,14.3,3.3,50.9,...,24.8,9.7,132.9,1019.4,48.4,24.1,217.9,18.7,7.0,Partially cloudy


---

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [7]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

ModuleNotFoundError: No module named 'hopsworks'

---

## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
        name = 'air_quality_fg',
        description = 'Air Quality characteristics of each day',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    

air_quality_fg.insert(df_air_quality)

#### <span style='color:#ff5f27'> 🌦 Weather Data

In [ ]:
weather_fg = fs.get_or_create_feature_group(
        name = 'weather_fg',
        description = 'Weather characteristics of each day',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    

weather_fg.insert(df_weather)

---